In [456]:
import pandas as pd
import numpy as np

In [457]:
df = pd.read_csv('path_detail.csv') #read the original dataset

#create the player column by merging the fnm and lnm
df['player'] = df['fnm'] + ' ' + df['lnm']
df = df.drop(columns=['fnm', 'lnm', 'ddst']) #drop the first names and lastnames (just keep the player column)

In [458]:
print(df.shape)
df = df.dropna() #drop all the NAs rows
print(df.shape)

(79776, 20)
(72254, 20)


In [459]:
pps = [] #a list that contains the amount of frames of each shooting motion
for element in df['pid'].unique():
    subdf = df[df['pid'] == element]
    pps.append(subdf.shape[0])
print(max(pps))
print(min(pps))

#the player with less amount of frams in hes motion has 287

589
287


In [460]:
#do an undersample that cant have more than 287 steps 
df_new = pd.DataFrame(columns = df.columns)
#for element in df['pid'].unique():
#    subdf = df[df['pid'] == element]
#    subdf = subdf.sample(n=15, random_state=1)
#    subdf = subdf.sort_values(by=['t'])
#    df_new = df_new.append(subdf)
s = 200 #the resample

for element in df['pid'].unique():
    subdf = df[df['pid'] == element]
    index_lst = []
    for step in range (s+1):
        
        index_lst.append(step * int((subdf.shape[0] - 1)/s))
    subdf = subdf.iloc[index_lst]
    subdf = subdf.sort_values(by=['t'])
    df_new = df_new.append(subdf)



df_new = df_new.drop(columns=['t'])
df = df_new

In [461]:
reorder_df = df_new[['pid','player','hght','rt']] #take the columns pid, player,hght and rt, and add them later to the new dataset
df_new = df_new.drop(columns=['pid','player','hght','rt'])

df_new_coor = df_new[['cx','cy','cz']]
df_new_vel = df_new[['cvx','cvy','cvz']]
df_new_acc = df_new[['cax','cay','caz']]
df_v_a = df_new[['cv','ca']]

In [462]:
#create the empty final datasets by iterating over the sample amounts and creating the column names
col_names = []
col_names_coor = []
col_names_vel = []
col_names_acc = []
col_names_va = []
for i in range (s+1):
    for name in list(df_new.columns):
        col_names.append(name + str(i))
        
    for name1 in list(df_new_coor.columns):
        col_names_coor.append(name1 + str(i))
        
    for name2 in list(df_new_vel.columns):
        col_names_vel.append(name2 + str(i))
        
    for name3 in list(df_new_acc.columns):
        col_names_acc.append(name3 + str(i))
        
    for name4 in list(df_v_a.columns):
        col_names_va.append(name4 + str(i))
        
        
final_df = pd.DataFrame(columns = col_names)    #one with all thevariables
final_coor = pd.DataFrame(columns = col_names_coor)    #one only with the coordinates
final_vel = pd.DataFrame(columns = col_names_vel)    #one only with the velocities
final_acc = pd.DataFrame(columns = col_names_acc)    #one only with the accelerations
final_va = pd.DataFrame(columns = col_names_va)    #one only with the absolut velocitie and acceleration

In [463]:
#iterate over the new_dfs to flatten them for each pid and add it to the final dataframes as a new row

for element in df['pid'].unique():
    subdf = df[df['pid'] == element]
    subdf = subdf[df_new.columns]
    flatten_df = subdf.stack().values
    final_df = final_df.append(pd.DataFrame(flatten_df.reshape(1,-1), columns=list(final_df)), ignore_index=True)
    
for element in df['pid'].unique():
    subdf = df[df['pid'] == element]
    subdf = subdf[df_new_coor.columns]
    flatten_df = subdf.stack().values
    final_coor = final_coor.append(pd.DataFrame(flatten_df.reshape(1,-1), columns=list(final_coor)), ignore_index=True)
    
for element in df['pid'].unique():
    subdf = df[df['pid'] == element]
    subdf = subdf[df_new_vel.columns]
    flatten_df = subdf.stack().values
    final_vel = final_vel.append(pd.DataFrame(flatten_df.reshape(1,-1), columns=list(final_vel)), ignore_index=True)
    
for element in df['pid'].unique():
    subdf = df[df['pid'] == element]
    subdf = subdf[df_new_acc.columns]
    flatten_df = subdf.stack().values
    final_acc = final_acc.append(pd.DataFrame(flatten_df.reshape(1,-1), columns=list(final_acc)), ignore_index=True)
    
for element in df['pid'].unique():
    subdf = df[df['pid'] == element]
    subdf = subdf[df_v_a.columns]
    flatten_df = subdf.stack().values
    final_va = final_va.append(pd.DataFrame(flatten_df.reshape(1,-1), columns=list(final_va)), ignore_index=True)
    

In [464]:

final_df = final_df.reset_index()

In [466]:
#keep only one ot the hghts, pid and player name for player
reorder_df = reorder_df.drop_duplicates(keep='first')
reorder_df = reorder_df.reset_index()

#and addit to the new dataframes
final = pd.concat([reorder_df, final_df], axis=1)
final_coor = pd.concat([reorder_df, final_coor], axis=1)
fina_vel = pd.concat([reorder_df, final_vel], axis=1)
final_acc = pd.concat([reorder_df, final_acc], axis=1)
final_va = pd.concat([reorder_df, final_va], axis=1)

In [468]:
#save them as CSVs 
final_coor.to_csv('final_coor.csv')
fina_vel.to_csv('fina_vel.csv')
final_acc.to_csv('final_acc.csv')
final_va.to_csv('final_va.csv')